# Splitting the dataset 

### Import Libraries

In [515]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import warnings

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

### Function to get the DataFrame for a specific driver and path order

In [516]:
def driver_dataframe(df, path_order, driver):
    """
    Extracts and processes the DataFrame for a specific driver and path order.
    
    Parameters:
    df (DataFrame): The original DataFrame.
    path_order (int): The path order identifier.
    driver (str): The driver class identifier.
    
    Returns:
    DataFrame: Processed DataFrame for the specified driver and path order.
    """
    driver_df = df[df['PathOrder'] == path_order]
    driver_df = driver_df[driver_df['Class'] == driver]
    driver_df = driver_df.sort_values(by='Time(s)')
    driver_df.drop(columns=['Class', 'PathOrder'], inplace=True)
    driver_df = driver_df.groupby('Time(s)').mean()
    driver_df['Time(s)'] = driver_df.index
    return driver_df

### Function to split the dataset

In [517]:
def split_dataset(df, train_ratio=0.8, random_seed=123):
    """
    Splits the dataset into training and testing sets based on driver and path order.
    
    Parameters:
    df (DataFrame): The original DataFrame.
    train_ratio (float): Ratio of the dataset to be used for training. Default is 0.8.
    random_seed (int): Seed for random number generator. Default is 123.
    
    Returns:
    DataFrame: Training set.
    DataFrame: Testing set.
    """
    
    # set a random seed 
    np.random.seed(random_seed)
    
    # create a copy of dataframe
    df_copy = df.copy()

    # Lists to hold individual DataFrames
    df_train = []
    df_test = []

    # Nested loops to process each path and driver
    for path in df_copy['PathOrder'].unique():
        for driver in df_copy['Class'].unique():
            driver_df = driver_dataframe(df_copy, path, driver)
            driver_df['PathOrder'] = path  # Add PathOrder
            driver_df['Class'] = driver    # Add Class

            # Calculate the index to split the data
            split_index = int(train_ratio * len(driver_df))

            # Split the data
            df_train_split = driver_df.iloc[:split_index]
            df_test_split = driver_df.iloc[split_index:]

            # Append the data to df_train and df_test lists
            df_train.append(df_train_split)
            df_test.append(df_test_split)

    # Concatenate the DataFrames
    df_train = pd.concat(df_train, ignore_index=True)
    df_test = pd.concat(df_test, ignore_index=True)

    return df_train, df_test

In [518]:
selected_features = ['Engine_soacking_time', 'Long_Term_Fuel_Trim_Bank1', 'Engine_coolant_temperature.1',
 'Torque_of_friction', 'Activation_of_Air_compressor', 'Intake_air_pressure', 'PathOrder', 'Accelerator_Pedal_value',
 'Master_cylinder_pressure', 'Vehicle_speed','Engine_coolant_temperature', 'Maximum_indicated_engine_torque',
 'Throttle_position_signal', 'Current_Gear', 'Calculated_LOAD_value', 'Engine_torque_after_correction',
 'Engine_torque', 'Class', 'Time(s)']

In [519]:
data = pd.read_csv("../data/raw/Driving Data(KIA SOUL)_(150728-160714)_(10 Drivers_A-J).csv")

In [520]:
selected_df = data[selected_features]

In [521]:
selected_df.head()

,Engine_soacking_time,Long_Term_Fuel_Trim_Bank1,Engine_coolant_temperature.1,Torque_of_friction,Activation_of_Air_compressor,Intake_air_pressure,PathOrder,Accelerator_Pedal_value,Master_cylinder_pressure,Vehicle_speed,Engine_coolant_temperature,Maximum_indicated_engine_torque,Throttle_position_signal,Current_Gear,Calculated_LOAD_value,Engine_torque_after_correction,Engine_torque,Class,Time(s)
0,3,-0.8,87,9.0,0,33,1,0.0,325.5,0,95,58.2,5.2,0,23.9,5.5,5.5,A,1
1,3,-0.8,87,7.8,0,40,1,0.0,0.9,0,95,53.9,6.1,0,30.6,7.0,7.0,A,2
2,3,-0.8,87,7.4,0,41,1,0.0,0.9,0,95,53.1,5.2,0,31.8,7.0,7.0,A,3
3,3,-0.8,87,6.6,0,38,1,0.0,0.9,0,95,53.1,4.7,0,29.0,7.0,7.0,A,4
4,3,-0.8,87,11.3,1,40,1,0.0,0.9,0,95,53.5,5.7,0,30.2,8.2,8.2,A,5


In [522]:
df_train, df_test = split_dataset(selected_df, 0.8)

In [523]:
df_train.shape

(37059, 19)

In [524]:
df_test.shape

(9277, 19)

In [525]:

def driver_dataframe_using_window_size(df, path_order, driver, window_size=60):
    """
    Extracts and processes the DataFrame for a specific driver and path order,
    and calculates rolling statistics with the specified window size.
    
    Parameters:
    df (DataFrame): The original DataFrame.
    path_order (int): The path order identifier.
    driver (str): The driver class identifier.
    window_size (int): The size of the rolling window.
    
    Returns:
    DataFrame: Processed DataFrame for the specified driver and path order with
               rolling statistics concatenated to the original features.
    """
    
    # Filter the DataFrame for the specific driver and path order
    driver_df = df[df['PathOrder'] == path_order]
    driver_df = driver_df[driver_df['Class'] == driver]
 
    
    # Drop the 'Class' and 'PathOrder' columns
    time = driver_df['Time(s)']
    driver_df.drop(columns=['Class', 'PathOrder', 'Time(s)'], inplace=True)
    

    # Calculate rolling statistics
    rolling_mean = driver_df.rolling(window=window_size).mean().add_suffix('_mean')
    rolling_median = driver_df.rolling(window=window_size).median().add_suffix('_median')
    rolling_std = driver_df.rolling(window=window_size).std().add_suffix('_std')
#     rolling_var = driver_df.rolling(window=window_size).var().add_suffix('_var')
#     rolling_min = driver_df.rolling(window=window_size).min().add_suffix('_min')
#     rolling_max = driver_df.rolling(window=window_size).max().add_suffix('_max')
    
    # Concatenate the original features with the rolling statistics
    # , rolling_var, rolling_min, rolling_max
    driver_df = pd.concat([driver_df, rolling_mean, rolling_median, rolling_std], axis=1)
    driver_df['Time(s)'] = time
    # Drop the first 'window_size' rows to make the concatenation match
    driver_df = driver_df.iloc[window_size-1:]
    
    return driver_df


In [526]:
def prepare_all_drivers(df, random_seed=123):
    """
    Prepares the dataset by processing each driver's data and calculating rolling statistics for each feature.
    
    Parameters:
    df (DataFrame): The original DataFrame.
    random_seed (int): The seed for the random number generator (default is 123).
    
    Returns:
    DataFrame: Processed DataFrame with rolling statistics and necessary columns for all drivers.
    """
    
    # Set the random seed for reproducibility
    np.random.seed(random_seed)
    
    # Create a copy of the original DataFrame to ensure the original data is not altered
    df_copy = df.copy()

    # List to hold individual DataFrames for each driver and path
    df_temp = []

    # Nested loops to process each path and driver
    for path in df_copy['PathOrder'].unique():
        for driver in df_copy['Class'].unique():
            # Process the DataFrame for the specific driver and path using window size
            driver_df = driver_dataframe_using_window_size(df_copy, path, driver)
            driver_df['PathOrder'] = path  # Add PathOrder column
            driver_df['Class'] = driver    # Add Class column

            # Append the processed DataFrame to the list
            df_temp.append(driver_df)

    # Concatenate all individual DataFrames into a single DataFrame
    all_drivers = pd.concat(df_temp, ignore_index=True)

    return all_drivers

In [527]:
df_train_preprocessed = prepare_all_drivers(df_train)
df_test_preprocessed = prepare_all_drivers(df_test)

In [528]:
df_train_preprocessed.head()

,Engine_soacking_time,Long_Term_Fuel_Trim_Bank1,Engine_coolant_temperature.1,Torque_of_friction,Activation_of_Air_compressor,Intake_air_pressure,Accelerator_Pedal_value,Master_cylinder_pressure,Vehicle_speed,Engine_coolant_temperature,Maximum_indicated_engine_torque,Throttle_position_signal,Current_Gear,Calculated_LOAD_value,Engine_torque_after_correction,Engine_torque,Engine_soacking_time_mean,Long_Term_Fuel_Trim_Bank1_mean,Engine_coolant_temperature.1_mean,Torque_of_friction_mean,Activation_of_Air_compressor_mean,Intake_air_pressure_mean,Accelerator_Pedal_value_mean,Master_cylinder_pressure_mean,Vehicle_speed_mean,Engine_coolant_temperature_mean,Maximum_indicated_engine_torque_mean,Throttle_position_signal_mean,Current_Gear_mean,Calculated_LOAD_value_mean,Engine_torque_after_correction_mean,Engine_torque_mean,Engine_soacking_time_median,Long_Term_Fuel_Trim_Bank1_median,Engine_coolant_temperature.1_median,Torque_of_friction_median,Activation_of_Air_compressor_median,Intake_air_pressure_median,Accelerator_Pedal_value_median,Master_cylinder_pressure_median,Vehicle_speed_median,Engine_coolant_temperature_median,Maximum_indicated_engine_torque_median,Throttle_position_signal_median,Current_Gear_median,Calculated_LOAD_value_median,Engine_torque_after_correction_median,Engine_torque_median,Engine_soacking_time_std,Long_Term_Fuel_Trim_Bank1_std,Engine_coolant_temperature.1_std,Torque_of_friction_std,Activation_of_Air_compressor_std,Intake_air_pressure_std,Accelerator_Pedal_value_std,Master_cylinder_pressure_std,Vehicle_speed_std,Engine_coolant_temperature_std,Maximum_indicated_engine_torque_std,Throttle_position_signal_std,Current_Gear_std,Calculated_LOAD_value_std,Engine_torque_after_correction_std,Engine_torque_std,Time(s),PathOrder,Class
0,129.0,0.0,61.0,20.50,1.0,49.5,1.00,1.00,16.5,62.0,67.20,8.45,2.5,38.6,23.85,23.85,129.0,-0.073333,59.733333,19.954167,0.933333,59.125000,1.562500,11.056667,3.925000,61.666667,63.302500,8.945000,5.566667,48.150000,25.255833,25.253333,129.0,0.0,59.5,20.5,1.0,59.5,0.0,1.95,2.00,61.5,61.15,8.95,7.0,48.425,26.65,26.55,0.0,0.289867,0.384737,2.605979,0.251549,10.812197,2.847756,30.029674,5.114639,0.447845,4.325598,1.904382,2.808080,10.501307,7.898869,7.977537,60,1,A
1,129.0,0.0,61.0,19.95,0.5,46.5,0.40,1.45,16.0,62.0,66.40,8.00,2.5,36.3,21.30,21.30,129.0,-0.066667,59.758333,20.075000,0.941667,58.983333,1.569167,8.300833,4.191667,61.658333,63.348333,8.952500,5.608333,48.016667,25.395833,25.435833,129.0,0.0,59.5,20.5,1.0,59.5,0.0,1.80,2.00,61.5,61.15,8.95,7.0,48.425,26.65,26.55,0.0,0.286810,0.416723,2.425771,0.227210,10.922286,2.844502,22.011446,5.319529,0.436586,4.343873,1.899675,2.741847,10.601330,7.749184,7.752906,61,1,A
2,129.0,0.0,61.0,17.20,0.5,45.5,1.95,1.00,15.0,62.0,65.80,8.00,2.5,35.1,20.50,20.50,129.0,-0.060000,59.783333,20.166667,0.950000,59.091667,1.601667,5.559167,4.441667,61.650000,63.354167,8.975833,5.650000,48.117500,25.636667,25.676667,129.0,0.0,59.5,20.5,1.0,59.5,0.0,1.70,2.00,61.5,61.15,8.95,7.0,48.425,26.65,26.55,0.0,0.283561,0.444997,2.197064,0.199576,10.751623,2.837401,7.676116,5.469555,0.424863,4.346977,1.878773,2.673314,10.445499,7.352123,7.354714,62,1,A
3,129.0,-0.4,61.0,16.40,0.5,56.0,8.40,1.00,14.5,62.5,67.40,11.75,2.5,46.1,28.15,27.95,129.0,-0.060000,59.808333,20.251667,0.958333,59.466667,1.741667,5.561667,4.683333,61.650000,63.403333,9.073333,5.691667,48.483333,26.005000,26.041667,129.0,0.0,59.5,20.5,1.0,59.5,0.0,1.70,2.00,61.5,61.15,8.95,7.0,48.425,26.85,26.75,0.0,0.283561,0.470230,1.930728,0.167019,10.223298,2.961550,7.674580,5.589008,0.424863,4.376167,1.868210,2.602300,9.965192,6.893510,6.893114,63,1,A
4,129.0,-0.4,61.0,16.20,0.5,67.0,10.75,1.15,14.5,62.5,69.15,32.65,2.5,59.4,33.20,33.20,129.0,-0.060000,59.833333,20.340000,0.966667,60.050000,1.920833,5.565833,4.925000,61.650000,63.507500,9.523333,5.733333,49.094167,26.447500,26.484167,129.0,0.0,59.5,20.5,1.0,61.0,0.0,1.70,2.25,61.5,61.15,8.95,7.0,49.675,26.95,27.15,0.0,0.283561,0.492887,1.586110,0.125774,9.609626,3.17208

In [529]:
df_train_preprocessed.shape

(35879, 67)

In [530]:
df_test_preprocessed.shape

(8097, 67)

In [531]:
X_train = df_train_preprocessed.drop(columns=['Class'])
y_train = df_train_preprocessed[['Class']]

X_test = df_test_preprocessed.drop(columns=['Class'])
y_test = df_test_preprocessed[['Class']]

### RandomForest Model

In [532]:
clf = RandomForestClassifier()

In [533]:
clf.fit(X_train, y_train)

RandomForestClassifier()

In [534]:
pred = clf.predict(X_test)

In [535]:
accuracy_score(y_test, pred)

0.8609361491910584

In [536]:
f1_score(pred, y_test, average='macro')

0.8544599539038741